In [ ]:
!pip install gensim==3.5.0
!pip install sentence-transformers
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.9 MB 1.4 MB/s 
  Created wheel for gensim: filename=gensim-3.5.0-cp38-cp38-linux_x86_64.whl size=24411786 sha256=7667167adbfbce85ba7cca376f56a730d82523311649ec164f9e9153c5c1b94a
  Stored in directory: /root/.cache/pip/wheels/f5/22/2d/bd6a60ceed1cbeb5e7eb7898e54acab40232515b66085176d5
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 5.8 MB 22.7 MB/s 
     |████████████████████████████████| 1.3 MB 60.6 MB/s 
     |████████████████████████████████| 182 kB 58.6 MB/s 
     |████████████████████████████████| 7.6 MB 51.1 MB/s 
  Create

In [ ]:
import string
import numpy as np
import pandas as pd
import math
import pickle
import numpy as np

from google.colab import drive

import nltk
nltk.download('popular')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

from scipy import sparse
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.word2vec import Word2Vec
from sentence_transformers import SentenceTransformer
import faiss

pd.set_option('display.max_columns', None)

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read final dataframe
food = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/256_data/food_sample.csv')

In [ ]:
# User input on website. Defines for now to show recommendations made by different algorithms
TEST_QUERY_INDEX = 9856
food.name.iloc[TEST_QUERY_INDEX]

'chicken   tortilla enchilada bake'

In [ ]:
# Tokenization

STOPWORDS = set(stopwords.words('english'))
MIN_CHAR = 3
MAX_CHAR = 150

# Tokenize text
def tokenizer(text, lemmatize):

  if lemmatize:
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(w) for w in word_tokenize(text)]
  else:
    tokens = [w for w in word_tokenize(text)]
  
  tokens_ = [w for w in tokens if (len(w) >= MIN_CHAR and len(w) < MAX_CHAR and w not in STOPWORDS)]
  
  return tokens_

## Search by recipe name

### Recipes with similar ingredients using TFIDF Vectorizer

In [ ]:
# Find similar rows using cosine similarity between test query and all other recipes
def extract_similar_rows(m, topk, mask = None):
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis = 0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] 

    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))

    mask = np.logical_or(cos_sim[index] != 0, mask)
    similar_row_indexes = index[mask][:topk]  
    
    return similar_row_indexes

# Returns index of recipes that are closest to search query
def get_reco_tfidf(sentence, tfidf_mat):

    tokens = [str(tok) for tok in tokenizer(sentence, True)]
    embed_tokens = vectorizer.transform(tokens)

    mat = cosine_similarity(embed_tokens, tfidf_mat)

    best_index = extract_similar_rows(mat, topk = 10)

    return best_index

In [ ]:
# TFIDF Vectorizer of ingredients

vectorizer = TfidfVectorizer() 
tfidf_mat = vectorizer.fit_transform(food['rec_ingredients'])

In [ ]:
test_name = food.name.iloc[TEST_QUERY_INDEX]
test_string = food[food.name == test_name]['rec_ingredients'].values[0]

# Get recommendation for searched recipe
similar_recipes = get_reco_tfidf(test_string, tfidf_mat)

display(food[['name', 'ingredients']].iloc[similar_recipes][1:])

,name,ingredients
144465,enchilada lasagna,corn tortillas cooked chicken enchilada sauce ...
198019,best easiest low fat chicken verde enchiladas,chicken breasts corn tortillas enchilada sauce...
190423,berdie s cheese enchilada casserole,corn tortillas red enchilada sauce mexican ble...
87598,speedy cheese and chicken enchiladas,onion olive oil cooked chicken green enchilada...
129775,cheese pork enchiladas,mexican blend cheese onion corn tortillas vege...
48732,easy enchiladas,ground beef enchilada sauce mexican blend chee...
173592,crock pot chicken enchilada,cooked chicken enchilada sauce cream of chicke...
141870,easy cheesy enchiladas,corn tortillas vegetable oil mexican blend che...
25638,skillet chicken cheese enchiladas,corn tortilla cooked chicken enchilada sauce m...


### Recipes with similar nutritional values using correlation matrix

In [ ]:
# Create pivot table

food_nutrition = food[['name', 'calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']]
food_nutrition_pivot = food_nutrition.pivot_table(columns = 'name', values = list(food_nutrition.columns)[1:])

In [ ]:
# Find correlation of searched recipe's nutrition values with other recipes
test_recipe = food.name.iloc[TEST_QUERY_INDEX]
item = food_nutrition_pivot[test_recipe]

# Find correlation
similar_items = food_nutrition_pivot.corrwith(item)
corr = pd.DataFrame(similar_items, columns = ['Correlation'])
corr.dropna(inplace = True)

In [ ]:
corr_top_20 = (corr['Correlation']> 0.999000).sort_values(ascending = False).iloc[:20]
recommended_recipe_id = food[food.name.isin(corr_top_20.index)]['recipe_id']
corr_top_20

name
hamburger and green bean casserole           True
crock pot chicken   cornbread dressing       True
packs a wallop beef stew                     True
morning breakfast panini                     True
spicy italian hero crescent ring             True
crock pot cheeseburger supper  so easy       True
everyone loves chicken casserole             True
wedding lasagna                              True
carnivore s lasagna                          True
spicy king ranch chicken                     True
golden crab puff                             True
sardines melt                                True
biscuit burgers  rsc                         True
sarita s chicken tortilla soup  muy bueno    True
sarma  cabbage rolls                         True
crock pot chicken ole                        True
hungarian szekely goulash                    True
carman s caper                               True
satay beef                                   True
weight watchers salisbury steak              

### Recipes with similar method of preparation using Sent2Vec embedding of steps

In [ ]:
# Instatiate sentence transformer model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
encodings_recipe = model.encode(food['steps'], show_progress_bar = True)

Batches:   0%|          | 0/6904 [00:00<?, ?it/s]

In [ ]:
# save embeddings

pickle.dump(encodings_recipe, open('/content/drive/MyDrive/Colab Notebooks/256_models/recipe_steps.pickle', 'wb'))

In [ ]:
encodings_recipe_steps_df = pd.DataFrame(pickle.load(open('/content/drive/MyDrive/Colab Notebooks/256_models/recipe_steps.pickle','rb')))

# Create sparse matrix of embeddings
encoding_sparse = sparse.csr_matrix(encodings_recipe_steps_df)

# Find cosine similarity
similarities_sparse = cosine_similarity(encoding_sparse[:5000])
df = pd.DataFrame(similarities_sparse)

data_similarity = df.unstack().reset_index() 
data_similarity.columns = ['recipe1', 'recipe2', 'cosine_similarity']

# Filter recipes that have cosine similarity below 1
data_similarity = data_similarity[data_similarity['cosine_similarity'] < 0.9999]

# Save data
data_similarity.to_csv('/content/drive/MyDrive/Colab Notebooks/256_data/steps_data_similarity.csv')

In [ ]:
data_similarity = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/256_data/steps_data_similarity.csv')

In [ ]:
recipe_dict = {}
for j,i in enumerate(food['name']):
    recipe_dict[j] = i

# Create recipe dict
data_similarity['recipe1_name'] = data_similarity['recipe1'].map(recipe_dict)
data_similarity['recipe2_name'] = data_similarity['recipe2'].map(recipe_dict)

# Find similarity rank between two recipes
data_similarity['similarity_rank'] = data_similarity.groupby(['recipe1'])['cosine_similarity'].rank('dense', ascending=False)
data_similarity = data_similarity[data_similarity['similarity_rank'] <= 10].reset_index(drop = True)

In [ ]:
def find_similar_dishes(list_names):
  for i in list_names:
      dummy_data =  data_similarity[data_similarity['recipe1_name'] == i]
      print ('Searched dish :',i)
      print ('Similar dishes')
      dummy_data.sort_values(inplace = True, by = ['similarity_rank']) 
      for j,i in enumerate(dummy_data['recipe2_name']):
          print (i)

In [ ]:
# Query strings to find recipes with similar method of preparation which will be inputted by the user

find_similar_dishes([food.name.iloc[654]])

Searched dish : lemon sugar cookies
Similar dishes
              great grandma s chocolate zucchini cake
              rice crispy chocolate chip oatmeal cookies
              peppermint candy crisps
              frosted ginger cookies
              go big red cake
              easy cheesecake tarts
              gobble them up oatmeal raisin cookies
              lemon blueberry tea bread
              black pepper cake
              glazed hazelnut chocolate torte


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


### Recipes from similar cuisine, similar nutrition, meal type and course type by finding Nearest Neighbours



In [ ]:
# Extract recipe features and create sparse matrix

recipe_features = food[['calories', 'total_fat', 'sugar','sodium', 'protein', 'saturated_fat', 'carbohydrates', 'healthy', 'unhealthy', \
                             'veg', 'non_veg', 'sweet', 'savory', 'cuisine_american', 'cuisine_asian', 'cuisine_australian', 'cuisine_european', 'cuisine_other', 'cuisine_indian']]

recipe_features_matrix = sparse.csr_matrix(recipe_features.values)

In [ ]:
# Nearest Neighbour algorithm

knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 10)
knn.fit(recipe_features_matrix)

distances, indices = knn.kneighbors(recipe_features.iloc[TEST_QUERY_INDEX, :].values.reshape(1, -1))

In [ ]:
# Print similar recipes

food[food.index.isin(indices.flatten())][['name']]

,name
9856,chicken tortilla enchilada bake
21808,award winning sea food chowder
78832,autumn eggs with potatoes
96229,turkey lasagna
128884,cheesy chicken rollups
133182,supper nachos
157473,jewel s ground beef casserole
163150,ham and cheese stuff n puff
167523,southwestern egg casserole
220200,spicy lasagna


## Search by ingredient 

### Recipes with similar ingredients and method of preparation using Word2Vec embeddings

In [ ]:
def is_word_in_model(word, model):
  # Check if searched ingredient is present in our vocab
  is_in_vocab = word in model.vocab
  return is_in_vocab

def predict_w2v(query_sentence, dataset, model, topk = 10):
  query_sentence = query_sentence.split()
  in_vocab_list, best_index = [], [0]*topk

  # create vocabulary list of query string
  for w in query_sentence:
    if is_word_in_model(w, model.wv):
      in_vocab_list.append(w)

  # find similarity between searched ingredients and all ingredients to return topk recipes
  if len(in_vocab_list) > 0:
    sim_mat = np.zeros(len(dataset))
    for i, data_sentence in enumerate(dataset):
        if data_sentence:
            sim_sentence = model.wv.n_similarity(in_vocab_list, data_sentence)
        else:
            sim_sentence = 0
        sim_mat[i] = np.array(sim_sentence)

    best_index = np.argsort(sim_mat)[::-1][:topk]
    
  return best_index

In [ ]:
# Word2Vec on ingredients and steps combined
word_arr = [s.split() for s in food['rec_ingredients_steps'].values]

# Word2Vec build vocabular and train on recipes data
word2vec_model = Word2Vec(min_count = 0, workers = 8) 
word2vec_model.build_vocab(word_arr)
word2vec_model.train(word_arr, total_examples = word2vec_model.corpus_count, epochs = 1)

In [ ]:
pickle.dump(word2vec_model, open('/content/drive/MyDrive/Colab Notebooks/256_models/word2vec_model.pickle', 'wb'))

In [ ]:
model = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/256_models/word2vec_model.pickle', 'rb'))

In [ ]:
# Predict and display most similar recipes in terms of ingredients and method of preparation

test_sentence = 'turkey sandwich cheese dal'
best_index = predict_w2v(test_sentence, word_arr, model)    
display(food[['name']].iloc[best_index])

,name
1624,turkish towel sandwich
69084,wasawich turkey and pepper jack
144655,pilgrim sandwich
101910,new york chicken burger
169960,exotic grilled cheese
84871,ham swiss roast beef and cheese wrap
90512,italian gut busters
30061,grilled gouda cheese sandwiches with smoked ha...
59192,turkey ranch and cheese snacks
34159,the tld sammy sandwiches


In [ ]:
# Predict on sample ingredients. Input will be given by user on wesbite
test_sentence = 'cake orange cream'
best_index = predict_w2v(test_sentence, word_arr, model)    
display(food[['name']].iloc[best_index])

,name
1076,4 points diet soda cake
177271,my version of a sunshine cake ww style
131570,just one more bite orange zucchini cake
71103,berries on a cloud
121790,basic trifle recipe
81850,raspberry lemon cream cake
129555,fresh orange cream cheese frosting
59690,no bake orange cheesecake
186236,strawberry lemon angel food trifle
109326,creamsicle milkshake


### Recipes with similar ingredients or similar cusine, type, steps, ingredients, etc. using Approximate Nearest Neighbor

In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
pickle.dump(model, open('/content/drive/MyDrive/Colab Notebooks/256_models/combined_model.pkl', 'wb'))

In [ ]:
model = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/256_models/combined_model.pkl', 'rb'))

In [ ]:
encodings_combined = model.encode(food['rec_combined'], show_progress_bar = True)

Batches:   0%|          | 0/6904 [00:00<?, ?it/s]

In [ ]:
# Hyperparameters
n_centroids = 20
m = 16

# Normalise embeddings so that the distance metric becomes cosine distance
faiss.normalize_L2(encodings_combined) 

# Create index on embeddings
quantiser = faiss.IndexFlatL2(encodings_combined.shape[1])
index = faiss.IndexIVFPQ (quantiser, encodings_combined.shape[1], n_centroids, m , 8)
index.train(encodings_combined) 

# Add embeddings to index along with recipe_ids 
ids = food['recipe_id'].tolist()
ids = np.array(ids)
index.add_with_ids(encodings_combined, ids)

In [ ]:
# Save index
faiss.write_index(index, '/content/drive/MyDrive/Colab Notebooks/256_models/combined.index')

In [ ]:
# Calculate L2 distance

def calculateInnerProduct(L2_score):
    return (2 - math.pow(L2_score, 2))/2

In [ ]:
index = faiss.read_index('/content/drive/MyDrive/Colab Notebooks/256_models/combined.index')

In [ ]:
# Search index and return topk recipes

def searchFAISSIndex(query, topk):    
    distance, indices = index.search(query, topk)

    ids = [i for i in indices][0]
    L2_score = [d for d in distance][0]

    inner_product = [calculateInnerProduct(l2) for l2 in L2_score]

    # Create resultant dataframe
    search_result = pd.DataFrame()
    search_result['recipe_id'] = ids

    # Compute distances
    search_result['cosine_sim'] = inner_product

    # Sort and return topk recipes based on cosine similarity
    distance = food[food['recipe_id'].isin(ids)]
    distance = pd.merge(distance, search_result, on = 'recipe_id')
    distance = distance.sort_values('cosine_sim', ascending = False)

    return distance

In [ ]:
query = 'american orange cake with frosting'

# Encode query using Sent2Vec
query_embedding = model.encode([query])[0]
# Reshape
query_embedding = query_embedding.reshape(1, query_embedding.shape[0])
# Normalise embedding
faiss.normalize_L2(query_embedding)

# Search index and return name of similar recipes
search_result = searchFAISSIndex(query_embedding, 10)

search_result['name'].values

array(['special occasion pumpkin roll aka hazel s pumpkin cake roll',
       'no bake orange cheesecake', 'chiffon pumpkin pie',
       'tiramisu  cooking light',
       'caramel drizzled butterscotch toffee crunch pie',
       'goat cheese cheesecake w caramel sauce   english walnuts',
       'amazing tiramisu', 'bread machine sourdough cinnamon rolls',
       'a new yorker s real italian cheesecake',
       'pistachio shortbread cookies'], dtype=object)